In [1]:
# -*- coding: utf-8 -*-
"""
Generated by ArcGIS ModelBuilder on : 2024-01-22 16:18:29
"""
import arcpy
from arcpy.ia import *
from sys import argv

In [2]:
GDB_C = r'C:\\Users\\wb527163\\GIS\\Mali_Cdrive\\Mali_Health_FY24\\Mali_Health_FY24_C\\Mali_Health_FY24_C.gdb\\'
GDB_temp = r'C:\\Users\\wb527163\\GIS\\Mali_Cdrive\\Mali_Health_FY24\\Mali_Health_FY24_C\\NF_pop.gdb\\'

GDB_B = r'C:\\Users\\wb527163\\GIS\\Mali_Cdrive\\Mali_Health_FY24\\Mali_Health_FY24_B.gdb\\'
GDB_A = r'C:\\Users\\wb527163\\GIS\\Mali_Cdrive\\Mali_Health_FY24\\Mali_Health_FY24.gdb\\'
GDB_R = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\'

In [3]:
arcpy.env.workspace  = GDB_R + "NF_RCA1.gdb"
rasters = arcpy.ListRasters("*", "")
print(rasters)

['NF_1_ACA', 'NF_2_ACA', 'NF_3_ACA', 'NF_4_ACA', 'NF_5_ACA', 'NF_6_ACA', 'NF_7_ACA', 'NF_8_ACA', 'NF_9_ACA', 'NF_10_ACA', 'NF_11_ACA', 'NF_12_ACA', 'NF_13_ACA', 'NF_14_ACA', 'NF_15_ACA', 'NF_16_ACA', 'NF_17_ACA', 'NF_18_ACA', 'NF_19_ACA', 'NF_20_ACA', 'NF_21_ACA', 'NF_22_ACA', 'NF_23_ACA', 'NF_24_ACA', 'NF_25_ACA', 'NF_26_ACA', 'NF_27_ACA', 'NF_28_ACA', 'NF_29_ACA', 'NF_30_ACA', 'NF_31_ACA', 'NF_32_ACA', 'NF_33_ACA', 'NF_34_ACA', 'NF_35_ACA']


In [4]:
rast = rasters[0]
print('"' + "'{}'".format(rast) + '"')

"'NF_1_ACA'"


In [10]:
print(GDB_R + "NF_RCA1.gdb" + '\\' + 'NF_33_ACA')

R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_RCA1.gdb\NF_33_ACA


In [13]:
def PeopleGainNewFacility(RCA = GDB_R + "NF_RCA1.gdb", ACA = GDB_R + "NF_ACA1.gdb"):  # Population stats for HF plus 1 NF

    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    # Check out any necessary licenses.
    arcpy.CheckOutExtension("spatial")
    arcpy.CheckOutExtension("ImageAnalyst")
    arcpy.CheckOutExtension("ImageExt")
    
    
    # Inputs
    Eligible_NF = GDB_C + "Eligible_NF"
    ADM1_Dissolve = GDB_C + "ADM1_Dissolve"
    wpop = arcpy.Raster(GDB_A + "wpop")
    pxHFj_RCA_Full = arcpy.Raster(GDB_C + "pxHFj_RCA")
    COMREF_MXmax = arcpy.Raster(GDB_B + "COMREF_MXmax")
    
    # Intermediate outputs
    pNF_ACA = GDB_temp + "pNF_ACA"
    pNF_ACA_tb = GDB_temp + "pNF_ACA_tb"
    pxHFj1_RCA = GDB_temp + "pxHFj1_RCA"
    pxHFj1_RCA_tb = GDB_temp + "pxHFj1_RCA_tb"
    pxHFj_RCA = GDB_temp + "pxHFj_RCA"
    pxHFj_RCA_tb = GDB_temp + "pxHFj_RCA_tb"
    pG20j1 = GDB_temp + "pG20j1"
    pG20j1_tb = GDB_temp + "pG20j1_tb"
    
    # Final storage of values
    NF2 = Eligible_NF + "_2"
    arcpy.management.CopyFeatures(Eligible_NF, NF2)
    
    
#     arcpy.env.workspace  = RCA
#     rasters = arcpy.ListRasters("*", "")

    for raster in rasters: 
        
        ACA_mask = GDB_R + "NF_ACA1.gdb" + '\\' + 'NF_33_ACA'
        RCA_selection = GDB_R + "NF_RCA1.gdb" + '\\' + raster
        expr = '"' + "'{}'".format(raster) + '"'

        # Process: Select Layer By Attribute (Select Layer By Attribute) (management)
        Eligible_NF_selection, Count = arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=Eligible_NF, 
            selection_type="NEW_SELECTION", 
            where_clause=f"NF_code = '{raster}' And pNF_ACA IS NULL", 
            invert_where_clause="")

        # Process: If Value Is (If Value Is) ()
        if Count is "0":
            print('Already calculated for ' + raster + '.')
        else:
            print('Running for %s. %s' % (raster, time.ctime()))

            
# # People in the New Facility catchment area

            # Process: RCalc_PplWithin6h (Raster Calculator) (ia)
            #RCalc_PplWithin6h = pNF_ACA
            with arcpy.EnvManager(mask=ACA_mask):
                pNF_ACA =  wpop
                #pNF_ACA.save(RCalc_PplWithin6h)
                
            print("Raster calculator finished: WorldPop within NF_ACA catchment. %s" % time.ctime())


            # Process: Zstat_PplWithin6h (Zonal Statistics as Table) (ia)
            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, 
                                                zone_field="OBJECTID", 
                                                in_value_raster=pNF_ACA, 
                                                out_table=pNF_ACA_tb, 
                                                ignore_nodata="DATA", 
                                                statistics_type="SUM", 
                                                process_as_multidimensional="CURRENT_SLICE", 
                                                percentile_values=90, 
                                                percentile_interpolation_type="AUTO_DETECT")#.save(zStat1)
            
            print("Zonal stats finished. %s" % time.ctime())
            
            # Process: Calculate Field (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=pNF_ACA_tb, 
                                            field="NF_code", 
                                            expression=expr, 
                                            expression_type="PYTHON3", 
                                            code_block="", 
                                            field_type="TEXT", 
                                            enforce_domains="NO_ENFORCE_DOMAINS")[0]
                
            #arcpy.management.AlterField(pxHFj1_RCA_tb, "SUM", "SUM1")
                
            # Process: Join Field (3) (Join Field) (management)
            arcpy.management.JoinField(in_data=NF2, 
                                       in_field="NF_code", 
                                       join_table=pNF_ACA_tb, 
                                       join_field="NF_code", 
                                       fields=["SUM"])[0]
            
            print("Joined to copy of New Facilities. %s" % time.ctime())

            # Process: Calculate Field (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=NF2, 
                                            field="pNF_ACA", 
                                            expression="'!SUM! where !SUM! is not None else !pNF_ACA!'", 
                                            expression_type="PYTHON3", 
                                            code_block="", 
                                            field_type="TEXT", 
                                            enforce_domains="NO_ENFORCE_DOMAINS")[0]
            
            print("Transferred to permanent attribute table. %s" % time.ctime())

            # Process: Delete Field (Delete Field) (management)
            arcpy.management.DeleteField(in_table=NF2, 
                                         drop_field=["SUM"], 
                                         method="DELETE_FIELDS")[0]

            
            print("Deleted temp field. %s" % time.ctime())
            
# # People-minutes in the New Facility catchment area

            # Process: RCalc_PplMin_NF (Raster Calculator) (ia)
            #RCalc_PplMin_NF = pxHFj1_RCA
            with arcpy.EnvManager(mask=ACA_mask):
                pxHFj1_RCA =  wpop * RCA_selection
                #pxHFj1_RCA.save(RCalc_PplMin_NF)
            
            print("Raster calculator finished: People-minutes with NF, within NF_ACA catchment. %s" % time.ctime())


            # Process: Zonal Statistics as Table (Zonal Statistics as Table) (ia)
            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, 
                                                zone_field="OBJECTID", 
                                                in_value_raster=pxHFj1_RCA, 
                                                out_table=pxHFj1_RCA_tb, 
                                                ignore_nodata="DATA", 
                                                statistics_type="SUM", 
                                                process_as_multidimensional="CURRENT_SLICE", 
                                                percentile_values=90, 
                                                percentile_interpolation_type="AUTO_DETECT")
                #.save(Zonal_Statistics_as_Table)

            print("Zonal stats finished. %s" % time.ctime())
            
            #arcpy.management.AlterField(pxHFj1_RCA_tb, 'SUM', 'SUM2')

            # Process: Calculate Field (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=pxHFj1_RCA_tb, 
                                            field="NF_code", 
                                            expression=expr, 
                                            expression_type="PYTHON3", 
                                            code_block="", 
                                            field_type="TEXT", 
                                            enforce_domains="NO_ENFORCE_DOMAINS")[0]

            # Process: Join Field (Join Field) (management)
            arcpy.management.JoinField(in_data=NF2, 
                                       in_field="NF_code", 
                                       join_table=pxHFj1_RCA_tb, 
                                       join_field="OBJECTID_1", 
                                       fields=["SUM"])[0]
            
            print("Joined to copy of New Facilities. %s" % time.ctime())

            # Process: Calculate Field (4) (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=NF2, 
                                            field="pxHFj1_RCA", 
                                            expression="'!SUM! if !SUM! is not None else !pxHFj1_RCA!'", 
                                            expression_type="PYTHON3", 
                                            code_block="", 
                                            field_type="TEXT", 
                                            enforce_domains="NO_ENFORCE_DOMAINS")[0]
            
            print("Transferred to permanent attribute table. %s" % time.ctime())

            # Process: Delete Field (4) (Delete Field) (management)
            arcpy.management.DeleteField(in_table=NF2, 
                                         drop_field=["SUM"], 
                                         method="DELETE_FIELDS")[0]
            
            print("Deleted temp field. %s" % time.ctime())

# # People-minutes in the New Facility catchment area before introducing New Facility

            # Process: RCalc_PplMin_HF (Raster Calculator) (ia)
            #RCalc_PplMin_HF = pxHFj_ACA
            with arcpy.EnvManager(mask= ACA_mask):
                pxHFj_ACA =  pxHFj_RCA_Full
                #pxHFj_ACA.save(RCalc_PplMin_HF)
                
            print("Raster calculator finished: People-minutes without NF, within NF_ACA catchment. %s" % time.ctime())


            # Process: Zonal Statistics as Table (4) (Zonal Statistics as Table) (ia)
            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, 
                                                zone_field="OBJECTID", 
                                                in_value_raster=pxHFj_ACA, 
                                                out_table=pxHFj_RCA_tb, 
                                                ignore_nodata="DATA", 
                                                statistics_type="SUM", 
                                                process_as_multidimensional="CURRENT_SLICE", 
                                                percentile_values=90, 
                                                percentile_interpolation_type="AUTO_DETECT")
                #.save(Zonal_Statistics_as_Table_4_)
            
            print("Zonal stats finished. %s" % time.ctime())
            
            # Process: Calculate Field (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=pxHFj_RCA_tb, 
                                            field="NF_code", 
                                            expression=expr, 
                                            expression_type="PYTHON3", 
                                            code_block="", 
                                            field_type="TEXT", 
                                            enforce_domains="NO_ENFORCE_DOMAINS")[0]

            #arcpy.management.AlterField(pxHFj_RCA_tb, 'SUM', 'SUM3')

            # Process: Join Field (4) (Join Field) (management)
            arcpy.management.JoinField(in_data=NF2, 
                                       in_field="NF_code", 
                                       join_table=pxHFj_RCA_tb, 
                                       join_field="OBJECTID_1", 
                                       fields=["SUM"])[0]
            
            print("Joined to copy of New Facilities. %s" % time.ctime())

            # Process: Calculate Field (3) (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=NF2, 
                                            field="pxHFj_RCA", 
                                            expression="'!SUM! if !SUM! is not None else !pxHFj_RCA!'", 
                                            expression_type="PYTHON3", 
                                            code_block="", 
                                            field_type="TEXT", 
                                            enforce_domains="NO_ENFORCE_DOMAINS")[0]
            
            print("Transferred to permanent attribute table. %s" % time.ctime())

            # Process: Delete Field (3) (Delete Field) (management)
            arcpy.management.DeleteField(in_table=NF2, 
                                         drop_field=["SUM"], 
                                         method="DELETE_FIELDS")[0]
            
            print("Deleted temp field. %s" % time.ctime())

# # People with reduction in travel time (20%+ gain in minutes saved).

            # Process: RCalc_PplGain20 (Raster Calculator) (ia)
            #RCalc_PplGain20 = pG20j1
            with arcpy.EnvManager(mask= ACA_mask):
                pG20j1 = Con( ((COMREF_MXmax - RCA_selection)/COMREF_MXmax) >= 0.2,  wpop)
                #pG20j1.save(RCalc_PplGain20)
            
            print("Raster calculator finished: People whose travel time is reduced, within NF_ACA catchment. %s" % time.ctime())

            # Process: Zonal Statistics as Table (2) (Zonal Statistics as Table) (ia)
            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, 
                                                zone_field="OBJECTID", 
                                                in_value_raster=pG20j1, 
                                                out_table=pG20j1_tb, 
                                                ignore_nodata="DATA", 
                                                statistics_type="SUM", 
                                                process_as_multidimensional="CURRENT_SLICE", 
                                                percentile_values=90, 
                                                percentile_interpolation_type="AUTO_DETECT")
                #.save(Zonal_Statistics_as_Table_2_)
            
            print("Zonal stats finished. %s" % time.ctime())
            
            # Process: Calculate Field (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=pG20j1_tb, 
                                            field="NF_code", 
                                            expression=expr, 
                                            expression_type="PYTHON3", 
                                            code_block="", 
                                            field_type="TEXT", 
                                            enforce_domains="NO_ENFORCE_DOMAINS")[0]
            
            #arcpy.management.AlterField(pG20j1_tb, 'SUM', 'SUM4')
            
            # Process: Join Field (2) (Join Field) (management)
            arcpy.management.JoinField(in_data=NF2, 
                                       in_field="NF_code", 
                                       join_table=pG20j1_tb, 
                                       join_field="OBJECTID_1", 
                                       fields=["SUM"])[0]
            
            print("Joined to copy of New Facilities. %s" % time.ctime())

            # Process: Calculate Field (2) (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=NF2, 
                                            field="pG20j1", 
                                            expression="'!SUM! if !SUM! is not None else !pG20j1!'", 
                                            expression_type="PYTHON3", 
                                            code_block="", 
                                            field_type="TEXT", 
                                            enforce_domains="NO_ENFORCE_DOMAINS")[0]
            
            print("Transferred to permanent attribute table. %s" % time.ctime())

            # Process: Delete Field (2) (Delete Field) (management)
            arcpy.management.DeleteField(in_table=NF2, 
                                         drop_field=["SUM"], 
                                         method="DELETE_FIELDS")[0]
            
            print("Deleted temp field. %s" % time.ctime())

In [ ]:
if __name__ == '__main__':
    # Global Environment settings
    with arcpy.EnvManager(scratchWorkspace=r"C:\Users\wb527163\GIS\Mali_Cdrive\Mali_Health_FY24\Mali_Health_FY24_C\Mali_Health_FY24_C.gdb", workspace=r"C:\Users\wb527163\GIS\Mali_Cdrive\Mali_Health_FY24\Mali_Health_FY24_C\Mali_Health_FY24_C.gdb"):
        PeopleGainNewFacility(*argv[1:])

Already calculated for NF_1_ACA.
Already calculated for NF_2_ACA.
Already calculated for NF_3_ACA.
Already calculated for NF_4_ACA.
Already calculated for NF_5_ACA.
Already calculated for NF_6_ACA.
Already calculated for NF_7_ACA.
Already calculated for NF_8_ACA.
Already calculated for NF_9_ACA.
Already calculated for NF_10_ACA.
Already calculated for NF_11_ACA.
Already calculated for NF_12_ACA.
Already calculated for NF_13_ACA.
Already calculated for NF_14_ACA.
Already calculated for NF_15_ACA.
Already calculated for NF_16_ACA.
Already calculated for NF_17_ACA.
Already calculated for NF_18_ACA.
Already calculated for NF_19_ACA.
Already calculated for NF_20_ACA.
Already calculated for NF_21_ACA.
Already calculated for NF_22_ACA.
Already calculated for NF_23_ACA.
Already calculated for NF_24_ACA.
Already calculated for NF_25_ACA.
Already calculated for NF_26_ACA.
Already calculated for NF_27_ACA.
Already calculated for NF_28_ACA.
Already calculated for NF_29_ACA.
Already calculated for 

In [ ]:
            # Process: Join Field (3) (Join Field) (management)
            NF2 = 
            NF_v2 = arcpy.management.JoinField(in_data=Eligible_NF_selection, 
                                               in_field="Join_Dummy", 
                                               join_table=pNF_ACA_tb, 
                                               join_field="OBJECTID_1", 
                                               fields=["SUM"])[0]

            # Process: Calculate Field (Calculate Field) (management)
            NF_v3 = arcpy.management.CalculateField(in_table=NF_v2, 
                                                    field="pNF_ACA", 
                                                    expression="!SUM!", 
                                                    expression_type="PYTHON3", 
                                                    code_block="", 
                                                    field_type="TEXT", 
                                                    enforce_domains="NO_ENFORCE_DOMAINS")[0]

            # Process: Delete Field (Delete Field) (management)
            NF_v4 = arcpy.management.DeleteField(in_table=NF_v3, 
                                                 drop_field=["SUM"], 
                                                 method="DELETE_FIELDS")[0]

            # Process: RCalc_PplMin_NF (Raster Calculator) (ia)
            RCalc_PplMin_NF = pxHFj1_RCA
            with arcpy.EnvManager(mask=ACA_mask):
                pxHFj1_RCA =  wpop* raster
                pxHFj1_RCA.save(RCalc_PplMin_NF)


            # Process: Zonal Statistics as Table (Zonal Statistics as Table) (ia)
            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, zone_field="OBJECTID", in_value_raster=pxHFj1_RCA, out_table=pxHFj1_RCA_tb, ignore_nodata="DATA", statistics_type="SUM", process_as_multidimensional="CURRENT_SLICE", percentile_values=90, percentile_interpolation_type="AUTO_DETECT")
                .save(Zonal_Statistics_as_Table)


            # Process: Join Field (Join Field) (management)
            NF_v5 = arcpy.management.JoinField(in_data=NF_v4, in_field="Join_Dummy", join_table=pxHFj1_RCA_tb, join_field="OBJECTID_1", fields=["SUM"])[0]

            # Process: Calculate Field (4) (Calculate Field) (management)
            NF_v6 = arcpy.management.CalculateField(in_table=NF_v5, field="pxHFj1_RCA", expression="!SUM!", expression_type="PYTHON3", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

            # Process: Delete Field (4) (Delete Field) (management)
            NF_v7 = arcpy.management.DeleteField(in_table=NF_v6, drop_field=["SUM"], method="DELETE_FIELDS")[0]

            # Process: RCalc_PplMin_HF (Raster Calculator) (ia)
            RCalc_PplMin_HF = pxHFj_ACA
            with arcpy.EnvManager(mask= ACA_mask):
                pxHFj_ACA =  "%pxHFj_RCA%"
                pxHFj_ACA.save(RCalc_PplMin_HF)


            # Process: Zonal Statistics as Table (4) (Zonal Statistics as Table) (ia)
            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, zone_field="OBJECTID", in_value_raster=pxHFj_ACA, out_table=pxHFj_RCA_tb, ignore_nodata="DATA", statistics_type="SUM", process_as_multidimensional="CURRENT_SLICE", percentile_values=90, percentile_interpolation_type="AUTO_DETECT")
                .save(Zonal_Statistics_as_Table_4_)


            # Process: Join Field (4) (Join Field) (management)
            NF_v8 = arcpy.management.JoinField(in_data=NF_v7, in_field="Join_Dummy", join_table=pxHFj_RCA_tb, join_field="OBJECTID_1", fields=["SUM"])[0]

            # Process: Calculate Field (3) (Calculate Field) (management)
            NF_v9 = arcpy.management.CalculateField(in_table=NF_v8, field="pxHFj_RCA", expression="!SUM!", expression_type="PYTHON3", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

            # Process: Delete Field (3) (Delete Field) (management)
            NF_v10 = arcpy.management.DeleteField(in_table=NF_v9, drop_field=["SUM"], method="DELETE_FIELDS")[0]

            # Process: RCalc_PplGain20 (Raster Calculator) (ia)
            RCalc_PplGain20 = pG20j1
            with arcpy.EnvManager(mask=r"R:\MLI\GEO\Team\Projects\MLI_Health_FY24_R\NF_ACA1.gdb\%Name%"):
                pG20j1 = Con( ((COMREF_MXmax - NF_1_ACA)/COMREF_MXmax) >= 0.2,  wpop)
                pG20j1.save(RCalc_PplGain20)


            # Process: Zonal Statistics as Table (2) (Zonal Statistics as Table) (ia)
            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, zone_field="OBJECTID", in_value_raster=pG20j1, out_table=pG20j1_tb, ignore_nodata="DATA", statistics_type="SUM", process_as_multidimensional="CURRENT_SLICE", percentile_values=90, percentile_interpolation_type="AUTO_DETECT")
                .save(Zonal_Statistics_as_Table_2_)


            # Process: Join Field (2) (Join Field) (management)
            NF_v11 = arcpy.management.JoinField(in_data=NF_v10, in_field="Join_Dummy", join_table=pG20j1_tb, join_field="OBJECTID_1", fields=["SUM"])[0]

            # Process: Calculate Field (2) (Calculate Field) (management)
            NF_v12 = arcpy.management.CalculateField(in_table=NF_v11, field="pG20j1", expression="!SUM!", expression_type="PYTHON3", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

            # Process: Delete Field (2) (Delete Field) (management)
            NF_v13 = arcpy.management.DeleteField(in_table=NF_v12, drop_field=["SUM"], method="DELETE_FIELDS")[0]